In [ ]:
import numpy as np
from scipy.linalg import eigh

In [ ]:
from scipy.sparse import csr_matrix, kron, eye, diags

def pauli_matrices():
    """
    pauli_matrices:
      Builds the Pauli matrices as sparse matrices.

    Returns
    -------
    s_x, s_y, s_z: tuple of scipy.sparse.csr_matrix
      Pauli matrices for a 2x2 system.
    """
    s_x = csr_matrix([[0, 1], [1, 0]])
    s_y = csr_matrix([[0, -1j], [1j, 0]])
    s_z = csr_matrix([[1, 0], [0, -1]])
    return s_x, s_y, s_z

def ising_hamiltonian(N, l):
    """
    ising_hamiltonian:
      Builds the Ising model Hamiltonian using sparse matrices.

    Parameters
    ----------
    N : int
      Number of spins.
    l : float
      Interaction strength.

    Returns
    -------
    H : scipy.sparse.csr_matrix
      Ising Hamiltonian.
    """

    dim = 2 ** N
    H_nonint = csr_matrix((dim, dim), dtype=np.complex128)
    H_int = csr_matrix((dim, dim), dtype=np.complex128)

    s_x, _, s_z = pauli_matrices()

    # Non-interacting term
    for i in range(N):
        zterm = kron(eye(2**i), kron(s_z, eye(2**(N - i - 1))), format="csr")
        H_nonint += zterm

    # Interaction term
    for i in range(N - 1):
        xterm = kron(eye(2**i), kron(s_x, kron(s_x, eye(2**(N - i - 2))), format="csr"), format="csr")
        H_int += xterm

    H = l * H_nonint + H_int
    return H


def truncate_basis(H, num_states):
    """Finds the lowest `num_states` eigenvectors of H and builds the projector."""
    eigenvalues, eigenvectors = eigh(H)

    low_eigenvectors = eigenvectors[:, :num_states]
    projector = sum(np.outer(low_eigenvectors[:, i], low_eigenvectors[:, i]) for i in range(num_states))
    
    return projector, eigenvalues[:num_states]


def update_hamiltonian(H, projector):
    """Projects the Hamiltonian into the truncated basis."""
    return (1/2)*np.dot(projector.T, np.dot(H, projector))

def update_operators(A, B, projector):
    """Projects the operators A and B into the truncated basis."""
    A_new = np.dot(projector.T, np.dot(A, projector))
    B_new = np.dot(projector.T, np.dot(B, projector))
    return A_new, B_new

def double_hamiltonian(H, A, B, N):
    
    H = np.kron(H, np.eye(2**N)) + np.kron(np.eye(2**N), H) + np.kron(A, B)
    
    return H

In [ ]:
# Manual implementation of RSRG
N_initial = 2  # Initial number of spins
lambd = 1.0    # Transverse field strength
dN = 4     # Number of states to keep in the truncated basis
max_iterations = 100
threshold = 1e-6

N = N_initial
H = ising_hamiltonian(N, lambd)
sigma_x, sigma_z, identity = pauli_matrices()
A = np.kron(np.eye(2**(N-1)), sigma_x)
B = np.kron(sigma_x, np.eye(2**(N-1)))
energy_density_prev = None

for iteration in range(max_iterations):
    H_2N = double_hamiltonian(H, A, B, N)

    # Compute lowest eigenvectors and truncate
    projector, eigenvalues = truncate_basis(H_2N, dN)
    energy_density = eigenvalues[0] / N

    # Check for convergence
    if energy_density_prev is not None:
        if abs(energy_density - energy_density_prev) < threshold:
            print(f"Converged at iteration {iteration}: Energy density = {energy_density}")
            break
    energy_density_prev = energy_density

    # Update Hamiltonian and system size
    H = update_hamiltonian(H_2N, projector)
    -----SONO ARRIVATO QUA A CONTROLLARE -----
    A, B = update_operators(A, B, projector)
    H = np.kron(H, np.eye(2)) + np.kron(np.eye(2**N), H) + np.kron(A, B)
    N *= 2
else:
    print("Did not converge within the maximum number of iterations.")

print(f"Final energy density: {energy_density}")
